# Recipe Recommender System

**Want a specific meal for within a certain budget for a specific number of people**

In [1]:
import pandas as pd
import numpy as np
import pickle

## Read in Files

### Grocery Similarity Model

In [2]:
grocery_similarities = pd.read_csv('../../data/05_model_output/data_matrix_sim.csv')

In [3]:
grocery_similarities.set_index('Unnamed: 0', inplace=True)

Let's take a look at our cosine similarity matrix.

In [4]:
grocery_similarities

,acorn squash,aduki beans,agave,albacore,alfalfa sprouts,alfredo,all,allspice,almond,almond butter,...,yahrzeit,yam,yamaimo,yan,yasai,year,yogurt kefir,yolk,zing,zinger babies
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
acorn squash,1.000000,0.001239,0.001198,0.001979,0.002560,0.000000,0.004003,0.002764,0.011173,0.007193,...,0.000000,0.003787,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
aduki beans,0.001239,1.000000,0.000000,0.000000,0.000000,0.000000,0.001002,0.000000,0.001427,0.002701,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
agave,0.001198,0.000000,1.000000,0.002396,0.000930,0.000910,0.000969,0.000000,0.009386,0.007838,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
albacore,0.001979,0.000000,0.002396,1.000000,0.000000,0.000000,0.010672,0.000000,0.005319,0.002877,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
alfalfa sprouts,0.002560,0.000000,0.000930,0.000000,1.000000,0.000778,0.002070,0.000000,0.006722,0.004783,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
alfredo,0.000000,0.000000,0.000910,0.000000,0.000778,1.000000,0.001622,0.000000,0.002887,0.002342,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
all,0.004003,0.001002,0.000969,0.010672,0.002070,0.001622,1.000000,0.003353,0.015676,0.007647,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
allspice,0.002764,0.000000,0.000000,0.000000,0.000000,0.000000,0.003353,1.000000,0.002388,0.005165,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.015530,0.000000
almond,0.011173,0.001427,0.009386,0.005319,0.006722,0.002887,0.015676,0.002388,1.000000,0.045738,...,0.000000,0.000873,0.000718,0.001708,0.000000,0.000000,0.0,0.000000,0.001708,0.001871


Because this was an item-item based similarity the output gives us items that are most like the other items. That means that when I put in parsley I get out other seasonings like the list shown below. In order to get things that go with ingredients we will need do do this by user-item similarity instead. Let's run this later. 

In [5]:
print(grocery_similarities.loc['parsley'].nlargest(11))

parsley     1.000000
garlic      0.161545
onion       0.159124
cilantro    0.153014
lemon       0.151143
tomatoes    0.128387
thyme       0.121335
dill        0.119339
onions      0.118740
small       0.117934
kale        0.117458
Name: parsley, dtype: float64


### Recipe Database

In [6]:
infile = open('../../data/03_processed/crf_ingred_dict','rb')
recipe_dict = pickle.load(infile)
infile.close()

In [7]:
infile = open('../../data/03_processed/crf_links_dict','rb')
recipe_links_dict = pickle.load(infile)
infile.close()

In [10]:
recipe_dict['grilled cheese blt']

[{'qty': '8', 'unit': 'slices', 'name': 'sourdough bread', 'comment': ''},
 {'qty': '4',
  'unit': 'tablespoon',
  'name': 'unsalted butter',
  'comment': 'at room temperature'},
 {'qty': '8',
  'unit': 'ounces 2 cups',
  'name': 'shredded cheddar cheese',
  'comment': ''},
 {'qty': '2',
  'unit': '',
  'name': 'slicing tomatoes',
  'comment': 'such as beefsteak Brandywine or Cherokee purple sliced 1 4 inch thick'},
 {'qty': '8', 'unit': 'to 12 slices', 'name': '', 'comment': ''},
 {'qty': 'cooked', 'unit': '', 'name': 'bacon', 'comment': ''},
 {'qty': '12',
  'unit': 'leaves',
  'name': 'butterhead or other crispy lettuce',
  'comment': ''}]

In [11]:
recipe_links_dict['grilled cheese blt']

'https://www.simplyrecipes.com/recipes/grilled_cheese_blt/'

### Grocery Prices 

In [ ]:
grocery_prices_marianos = pd.read_csv('../../data/03_processed/grocery_prices_marianos_final.csv')

In [ ]:
grocery_prices_marianos

***

In [ ]:
def extract_ingredients(single_recipe_ing_list):
    ret = []
    for ingredient in single_recipe_ing_list:
        ret.append(ingredient['name'])
    return ret 

In [ ]:
def find_recipe_by_ingredient(recipe_dict, search_item):
    
    listofkeys = []
    listofitems = recipe_dict.items()

    for item in listofitems:
        lst = extract_ingredients(item[1])
        subs = search_item
        if any(subs in s for s in lst):
            listofkeys.append(item[0])
    return listofkeys

In [ ]:
print(grocery_similarities.loc['parsley'].nlargest(11))

In [ ]:
find_recipe_by_ingredient(recipe_dict, 'chocolate')

In [ ]:
recipe_dict['cincinnati chili']

## Recommendor System sudo-code

```python


```